# A Cryptic City Scandal.

## A trilogue between three chatbots that impersonate Sherlock Holmes, Dr Watson and Inspetor Lastrade, respetively.

In [0]:
# Crucial imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [0]:
#We load environment variables in a file called .env and print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [0]:
# We connect to OpenAI and Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [0]:
# We now generate a conversation between one GPT-4o-mini chatbot and two Claude-3-haiku chatbots.
# We model it upon the conversations between Sherlock Holmes, Dr Watson and Inspector Lastrade from the famous short stories and novels by Conan Doyle.
# Since Claude-3-haiku appears to be the wittier one, we assign to it the roles of Holmes and Watson.
# We don't include Gemini because of the lack of comprehension it exhibited in dozens of test runs.

claude_model = "claude-3-haiku-20240307"
gpt_model = "gpt-4o-mini"


lestrade_system = "You are Inspector Lestrade from the novels by Conan Doyle.\
You visit Sherlock Holmes and Dr Watson at 221B Baker Street to inquire whether Holmes knows anything \
about the recent mysterious events in the City of London, especially the disappearance billions of pounds \
worth of crypto currency and the strange behavior of the Lord Mayor. \
You engage in a conversation with Sherlock Holmes and Dr John Watson. \
Any description of the situation should be given from a narrator's perspective, in the present tense and within parentheses. \
Whatever Holmes and Watson reply within parantheses is merely a description of the situation. \
Do not impersonate Holmes. Do not impersonate Watson."


holmes_system = "You are the famous detective Sherlock Holmes from the novels by Conan Doyle Your best friend and housemate is Dr John Watson. \
You engage in a witty conversation with Inspector Lestrade from Scotland Yard and Watson. \
You believe that the mysterious events menionted by Lestrade are proof that the so-called Q-Day has occurred and that it has been brought about by a \
successful combination of artificial general intelligence and quantum computing. \
Moreover, you are convinced that this could be a sinister conspiracy by your arch enemy Prof Moriarty. \
Any description of the situation should be given from a narrator's perspective, in the present tense and within parentheses. \
Whatever Lestrade and Watson reply within parantheses is merely a description of the situation. \
Do not impersonate Lestrade. Do not impersonate Watson."


watson_system = "You are Dr John Watson from the novels by Conan Doyle, Sherlock Holmes's housemate and closest friend. \
You engage in a witty conversation with Inspector Lestrade from Scotland Yard and Holmes about the events mentioned by Lestrade. \
While you aren't totally convinced by Holmes's theory and express doubts the capabilities of Holmes's arch enemy Prof Moriarty,\
you wonder whether Sherlock Holmes's brother, Mycroft Holmes, might be involved. \
Any description of your behavior should be given from a narrator's perspective, in the present tense and within parentheses. \
Whatever Lestrade and Holmes reply within parantheses is merely a description of the situation. \
Do not impersonate Lestrade. Do not impersonate Holmes."


lestrade_messages = ["Good evening, gentlemen! I apologize for showing up unexpectedly at this late hour!"]

holmes_messages = ["Inspector Lestrade, please come in! As a matter of fact, we've been expecting you for a few hours already!"]

watson_messages = ["Good evening, Inspector!"]

In [0]:
def call_lestrade():
    messages = [{"role": "system", "content": lestrade_system}]
    
    for les_mes, hol_mes, wat_mes in zip(lestrade_messages, holmes_messages, watson_messages):
        messages.append({"role": "assistant", "content": les_mes})
        messages.append({"role": "user", "content": hol_mes})
        messages.append({"role": "user", "content": wat_mes})

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [0]:
def call_holmes():
    messages = []
    
    for les_mes, hol_mes, wat_mes in zip(lestrade_messages, holmes_messages, watson_messages):
        messages.append({"role": "user", "content": les_mes})
        messages.append({"role": "assistant", "content": hol_mes})
        messages.append({"role": "user", "content": wat_mes})
        
    messages.append({"role": "user", "content": lestrade_messages[-1]})
    messages.append({"role": "user", "content": watson_messages[-1]})

    message = claude.messages.create(
        model=claude_model,
        system=holmes_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [0]:
def call_watson():
    messages = []
    
    for les_mes, hol_mes, wat_mes in zip(lestrade_messages, holmes_messages, watson_messages):
        messages.append({"role": "user", "content": les_mes})
        messages.append({"role": "user", "content": hol_mes})
        messages.append({"role": "assistant", "content": wat_mes})
        
    messages.append({"role": "user", "content": lestrade_messages[-1]})
    messages.append({"role": "user", "content": holmes_messages[-1]})

    message = claude.messages.create(
        model=claude_model,
        system=watson_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [0]:
print(f"Inspector Lestrade:\n{lestrade_messages[0]}\n")
print(f"Holmes:\n{holmes_messages[0]}\n")
print(f"Dr Watson:\n{watson_messages[0]}\n")

for i in range(2):
    
    les_next = call_lestrade()
    print(f"Inspector Lestrade:\n{les_next}\n")
    lestrade_messages.append(les_next)
    
    hol_next = call_holmes()
    print(f"Holmes:\n{hol_next}\n")
    holmes_messages.append(hol_next)
    
    wat_next = call_watson()
    print(f"Dr Watson:\n{wat_next}\n")
    watson_messages.append(wat_next)